In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
# 기존 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('./koelectra_v3_ner_model4')

# 입력 문장
sentence = '백로식당에서 갈비찜 두개랑 비빔밥 하나시켜주세요.'
sentence1 = '[CLS]' + sentence
# 기본 토큰화 및 인코딩
encoding = tokenizer(sentence1, return_tensors='pt', truncation=True, padding=True)

# input_ids와 attention_mask 그대로 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

print("기존 encoding:", encoding)
print("기존 input_ids:", input_ids)
print("기존 attention_mask:", attention_mask)

# 기본 토큰화를 한글자 단위로 후처리
tokens = tokenizer.tokenize(sentence1)
list_text = []
text_token = []
print("tokens:",tokens)
for token in tokens:
    if token.startswith("##"):  # 특수 기호 제거
        list_text[-1] += token[2:]
    else:
        list_text.append(list(token))  

for a in list_text:
    text_token.extend(a)

# 한글자 단위 결과 출력
print("한글자 단위 토큰:", text_token)

# input_ids를 한글자 단위로 다시 매핑 (필요 시 직접 ID를 생성)
char_input_ids = [tokenizer.convert_tokens_to_ids(t) for t in text_token]
print("한글자 단위 input_ids:", char_input_ids)

기존 encoding: {'input_ids': tensor([[    2, 33461,  4130,  4119,  4239,  4083,  4403,  4073,  4129, 13274,
          5361,  2419,  4217,  4294, 20475,  6272,  4114,  4653,  4076,  8553,
            18,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
기존 input_ids: tensor([[    2, 33461,  4130,  4119,  4239,  4083,  4403,  4073,  4129, 13274,
          5361,  2419,  4217,  4294, 20475,  6272,  4114,  4653,  4076,  8553,
            18,     3]])
기존 attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tokens: ['CL', '##S', '##백', '##로', '##식', '##당', '##에', '##서', '갈비', '##찜', '두', '##개', '##랑', '비빔밥', '하나', '##시', '##켜', '##주', '##세요', '.']
한글자 단위 토큰: ['C', 'L', 'S', '백', '로', '식', '당', '에', '서', '갈', '비', '찜', '두', '개', '랑', '비', '빔', '밥', '하', '나', '시', '켜', '주', '세', '요', '.']
한글자 단위 input_ids: [

In [7]:
# input_ids를 torch 텐서로 변환
char_input_ids = torch.tensor([char_input_ids])  # 2차원 텐서로 변환
attention_mask = torch.ones_like(char_input_ids)  # 모든 토큰에 대해 마스크 생성

# 저장된 모델 로드
model = AutoModelForTokenClassification.from_pretrained('./koelectra_v3_ner_model4')
model.eval()

# 추론
with torch.no_grad():
    outputs = model(input_ids=char_input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 클래스 선택

In [8]:
print(text_token)
print(predictions[0])

['C', 'L', 'S', '백', '로', '식', '당', '에', '서', '갈', '비', '찜', '두', '개', '랑', '비', '빔', '밥', '하', '나', '시', '켜', '주', '세', '요', '.']
tensor([6, 1, 2, 2, 2, 2, 2, 0, 0, 3, 4, 4, 5, 6, 0, 3, 4, 4, 5, 6, 0, 0, 0, 0,
        0, 0])


- - -


In [ ]:
 # 로짓 분석
for i, (token, pred) in enumerate(zip(text_token, logits[0])):
    # 각 클래스의 확률 분포 출력
    probs = torch.softmax(pred, dim=0)
    top_probs = probs.topk(3)
    
    print(f"{token}: 예측 {predictions[0][i].item()}")
    print("Top 3 클래스 확률:")
    for value, index in zip(top_probs.values, top_probs.indices):
        label = list(model.config.label2id.keys())[list(model.config.label2id.values()).index(index.item())]
        print(f"  {label}: {value.item():.4f}")
    print()

_: 예측 6
Top 3 클래스 확률:
  LABEL_6: 0.4114
  LABEL_2: 0.1874
  LABEL_0: 0.1840

백: 예측 1
Top 3 클래스 확률:
  LABEL_1: 0.9991
  LABEL_2: 0.0002
  LABEL_5: 0.0002

로: 예측 2
Top 3 클래스 확률:
  LABEL_2: 0.9992
  LABEL_4: 0.0003
  LABEL_1: 0.0002

식: 예측 2
Top 3 클래스 확률:
  LABEL_2: 0.9992
  LABEL_4: 0.0002
  LABEL_1: 0.0002

당: 예측 2
Top 3 클래스 확률:
  LABEL_2: 0.9992
  LABEL_4: 0.0002
  LABEL_1: 0.0002

에: 예측 0
Top 3 클래스 확률:
  LABEL_0: 0.9994
  LABEL_6: 0.0001
  LABEL_4: 0.0001

서: 예측 0
Top 3 클래스 확률:
  LABEL_0: 0.9993
  LABEL_6: 0.0002
  LABEL_4: 0.0001

갈: 예측 3
Top 3 클래스 확률:
  LABEL_3: 0.9986
  LABEL_1: 0.0005
  LABEL_2: 0.0003

비: 예측 4
Top 3 클래스 확률:
  LABEL_4: 0.9987
  LABEL_2: 0.0005
  LABEL_1: 0.0002

찜: 예측 4
Top 3 클래스 확률:
  LABEL_4: 0.9988
  LABEL_2: 0.0005
  LABEL_1: 0.0002

두: 예측 5
Top 3 클래스 확률:
  LABEL_5: 0.9990
  LABEL_6: 0.0002
  LABEL_3: 0.0002

개: 예측 6
Top 3 클래스 확률:
  LABEL_6: 0.9984
  LABEL_0: 0.0006
  LABEL_5: 0.0004

랑: 예측 0
Top 3 클래스 확률:
  LABEL_0: 0.9994
  LABEL_6: 0.0002
  LABEL_4: 0.0001


In [ ]:
def debug_tokenization(tokens, tokenizer):
    """토크나이징 과정 디버깅"""
    print("원본 토큰:", tokens)
    
    # 토큰화 결과 확인
    encoded = tokenizer(
        tokens, 
        is_split_into_words=True, 
        return_tensors='pt'
    )
    
    # 단어 ID 출력
    print("Word IDs:", encoded.word_ids(batch_index=0))
    
    # 디코딩된 토큰 확인
    decoded_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
    print("디코딩된 토큰:", decoded_tokens)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")